# 1. Sorri not veri gud in inglish

Have you ever googled someone's name without knowing exactly how it should be written? Were you ever reluctant to look up the correct spelling of a query you typed? Or just unable to type properly because of being in a rush? Modern search engines usually do a pretty good job in deciphering defective user input. In order to be able to do that, a good spell-checking mechanism should be incorporated into a search procedure. Today we will take one step further towards building a good search engine and work on tolerant retrieval with respect to user queries. We will consider two cases:

1. Users know that they don't know the correct spelling OR they want to get the results that follow some known pattern, so the use so called wildcards - queries like `retr*val`;
2. Users don't know the correct spelling OR they don't care OR they are in a rush OR they expect that mistakes will be corrected OR /your option/... so they make mistakes and we need to handle them using:

    2.1. Trigrams with Jaccard coefficient;
    
    2.2. Simple spellchecker by Peter Norvig with QWERTY weights;

## 1.1. Handling wildcards

We will handle wildcard queries using k-grams. K-gram is a list of consecutive k chars in a string - i.e., for the word *'star'*, it will be '*\$st*', '*sta*', '*tar*', and '*ar$*', if we take k=3. Take a look at the [book](https://nlp.stanford.edu/IR-book/pdf/irbookonlinereading.pdf)'s *chapter 3.2.2* to understand how k-grams can help efficiently match a wildcard against dictionary words. Here we will only consider wildcards with star symbols (may be multiple).

Notice that for building k-grams index, **we will need a vocabulary of original (correct) word forms** to compare words in user input to the vocabulary of "correct" words (think why inverted index which we built for stemmed words doesn't work here).   

You need to implement the following:

- `build_inverted_index_orig_forms` - creates inverted index of original world forms from `facts` list, which is already given to you.  
    Output format: `term:[collection_frequency, (doc_id_1, doc_freq_1), (doc_id_2, doc_freq_2), ...]`
    

- `build_k_gram_index` - creates k-gram index which maps every k-gram encountered in facts collection to a list of words containing this k-gram. Use the abovementioned inverted index of original words to construct this index.  
    Output format: `'k_gram': ['word1_with_k_gram', 'word2_with_k_gram', ...]`
    
    
- `generate_wildcard_options` - produce a list of vocabulary words matching given wildcard by intersecting postings of k-grams present in the wildcard (refer to *ch 3.2.2*). 

- `search_wildcard` - return list of facts that contain the words matching a wildcard query.


We will use the dataset with curious facts for testing.

### 1.1.1. Downloading the dataset

In [3]:
import urllib.request
data_url = "https://raw.githubusercontent.com/IUCVLab/information-retrieval/main/datasets/facts.txt"
local_filename, headers = urllib.request.urlretrieve(data_url)

facts = []
with open(local_filename) as fp:
    for cnt, line in enumerate(fp):
        facts.append(line.strip('\n'))
        
print(*facts[-15:], sep='\n')

141. Months that begin on a Sunday will always have a "Friday the 13th."
142. The placement of a donkey's eyes in its' heads enables it to see all four feet at all times!
143. Some worms will eat themselves if they can't find any food!
144. Dolphins sleep with one eye open!
145. It is impossible to sneeze with your eyes open.
146. In France, it is legal to marry a dead person.
147. Russia has a larger surface area than Pluto.
148. There's an opera house on the U.S.-Canada border where the stage is in one country and half the audience is in another.
149. The harder you concentrate on falling asleep, the less likely to fall asleep.
150. You can't hum while holding your nose closed.
151. Women have twice as many pain receptors on their body than men. But a much higher pain tolerance.
152. There are more stars in space than there are grains of sand on every beach in the world.
153. For every human on Earth there are 1.6 million ants.
154. The total weight of all those ants, however, is abo

### 1.1.2. Implementation of search

In [4]:
import nltk
from collections import Counter
import re


def build_inverted_index_orig_forms(documents):
    #TODO build an inverted index of original word forms 
    # (without stemming, just word tokenized and lowercased)   
    inverted_index = {}
    for i, doc in enumerate(documents):
        tokens = nltk.word_tokenize(doc.lower())
        file_index = Counter(tokens)
        # update global index
        for term in file_index.keys():
            file_freq = file_index[term]
            if term not in inverted_index:                
                inverted_index[term] = [file_freq, (i, file_freq)]
            else:
                inverted_index[term][0] += file_freq
                inverted_index[term].append((i, file_freq))
    
    return inverted_index


def split_to_k_grams(word, k):
    return [word[i:i + k] for i in range(len(word) - k + 1)]


def build_k_gram_index(inverted_index, k):
    k_gram_index = {}
    for word in inverted_index.keys():
        augmented_w = '$' + word + '$'
        k_grams = split_to_k_grams(augmented_w, k)
        for gram in k_grams:
            if gram in k_gram_index:
                k_gram_index[gram].append(word)
            else:
                k_gram_index[gram] = [word]
    return k_gram_index


def generate_wildcard_options(wildcard, k_gram_index, inverted_index):
    augmented_wc = '$' + wildcard + '$'
    split_wc = augmented_wc.split("*")
    k_grams = set()
    k = len(list(k_gram_index.keys())[0])
    # forming list of k_grams for the query
    for split in split_wc:
        if len(split) >= k:
            k_grams.update(split_to_k_grams(split,k))
    sorted_postings = []
    # retrieve postings list for each k-gram
    for k_gram in k_grams:
        postings = set(k_gram_index[k_gram])
        sorted_postings.append((len(postings), postings))
    sorted_postings.sort()
    if not sorted_postings:
        return []
    # take the first (shortest) posting list and iteratively intersect with next ones
    corrections = set(sorted_postings[0][1])
    for i in range(1, len(sorted_postings)):
        corrections = set.intersection(corrections, sorted_postings[i][1])
    # remove not matching ones
    regexp = re.compile(wildcard.replace('*', '\w*'))
    filtered_corrections = []
    for correction in corrections:
        if re.search(regexp, correction) is not None:
            filtered_corrections.append(correction)
            
    sorted_filtered_corrections = sorted(filtered_corrections, key= lambda x: inverted_index.get(x)[0], reverse=True)
    return sorted_filtered_corrections


def search_wildcard(wildcard, k_gram_index, index, docs):
    wildcard_options = generate_wildcard_options(wildcard, k_gram_index, index)
    doc_ids = set()
    for term in wildcard_options:        
        posting = index[term][1:]
        # extract document info only
        doc_ids.update([i[0] for i in posting])
    results = []
    for doc_id in doc_ids:
        results.append(docs[doc_id])
       
    return results

### 1.1.3. Tests

In [5]:
index_orig_forms = build_inverted_index_orig_forms(facts)
k_gram_index = build_k_gram_index(index_orig_forms, 3)
k_gram_index
index_orig_forms

{'1.': [1, (0, 1)],
 'if': [11,
  (0, 1),
  (9, 1),
  (16, 1),
  (33, 1),
  (53, 1),
  (90, 1),
  (134, 1),
  (135, 1),
  (137, 2),
  (146, 1)],
 'you': [29,
  (0, 2),
  (4, 2),
  (9, 3),
  (17, 1),
  (32, 2),
  (33, 1),
  (53, 2),
  (70, 1),
  (79, 1),
  (86, 1),
  (87, 1),
  (93, 3),
  (126, 1),
  (127, 2),
  (135, 1),
  (137, 2),
  (139, 1),
  (152, 1),
  (153, 1)],
 'somehow': [1, (0, 1)],
 'found': [4, (0, 1), (42, 1), (68, 1), (132, 1)],
 'a': [89,
  (0, 2),
  (2, 1),
  (7, 2),
  (9, 1),
  (10, 1),
  (11, 1),
  (13, 1),
  (14, 1),
  (19, 3),
  (22, 1),
  (23, 1),
  (25, 1),
  (27, 1),
  (30, 2),
  (31, 2),
  (33, 1),
  (34, 1),
  (36, 1),
  (38, 1),
  (42, 1),
  (46, 1),
  (48, 1),
  (49, 2),
  (53, 2),
  (54, 1),
  (56, 1),
  (58, 2),
  (59, 1),
  (61, 1),
  (63, 1),
  (64, 1),
  (65, 1),
  (66, 2),
  (68, 2),
  (77, 2),
  (79, 1),
  (81, 1),
  (82, 1),
  (83, 2),
  (91, 1),
  (94, 2),
  (98, 2),
  (102, 2),
  (105, 1),
  (109, 1),
  (115, 1),
  (116, 1),
  (119, 1),
  (120, 2),

In [6]:
index_orig_forms = build_inverted_index_orig_forms(facts)
k_gram_index = build_k_gram_index(index_orig_forms, 3)

wildcard = "*re"

wildcard_options = generate_wildcard_options(wildcard, k_gram_index, index_orig_forms)
print(wildcard_options)
assert(len(wildcard_options) >= 3)
assert("red" not in wildcard_options) 

wildcard_results = search_wildcard(wildcard, k_gram_index, index_orig_forms, facts)
# some pretty printing
for r in wildcard_results:
    # highlight terms for visual evaluation
    for term in wildcard_options:
        r = re.sub(r'(' + term + ')', r'\033[1m\033[91m\1\033[0m', r, flags=re.I)
    print(r)

assert(len(wildcard_results) >=3)

assert "13. James Buchanan, the 15th U.S. president continuously bought slaves with his own money in order to free them." in search_wildcard("pres*dent", k_gram_index, index_orig_forms, facts)
assert "40. 9 out of 10 Americans are deficient in Potassium." in search_wildcard("p*tas*um", k_gram_index, index_orig_forms, facts)
assert "61. A man from Britain changed his name to Tim Pppppppppprice to make it harder for telemarketers to pronounce." in search_wildcard("*price", k_gram_index, index_orig_forms, facts)

['are', 'more', 'there', 'were', 'before', 'where', 'furniture', 'measure', 'core', 'store', 'snore', 'rupture', 'wore', 'millionaire', "'re", 'empire']
1. If you somehow found a way to extract all of the gold from the bubbling core of our lovely little planet, you would be able to cover all of the land in a layer of gold up to your knees.
127. 68% of the universe is dark energy, and 27% is dark matter; both are invisible, even with our powerful telescopes. This means we have only seen 5% of the universe from earth.
128. The founders of Google were willing to sell Google for $1 million to Excite in 1999, but Excite turned them down. Google is now worth $527 Billion.
5. You burn more calories sleeping than you do watching television.
6. There are more lifeforms living on your skin than there are people on the planet.
130. There are 60,000 miles of blood vessels in the human body.
7. Southern sea otters have flaps of skin under their forelegs that act as pockets. When diving, they use th

## 1.2. Handling typos

### 1.2.0. Dataset 

Download github typo dataset from [here](https://github.com/mhagiwara/github-typo-corpus).
Load it with this code:

In [7]:
!pip install jsonlines

DEPRECATION: Configuring installation scheme with distutils config files is deprecated and will no longer work in the near future. If you are using a Homebrew or Linuxbrew Python, please see discussion at https://github.com/Homebrew/homebrew-core/issues/76621
DEPRECATION: Configuring installation scheme with distutils config files is deprecated and will no longer work in the near future. If you are using a Homebrew or Linuxbrew Python, please see discussion at https://github.com/Homebrew/homebrew-core/issues/76621

[notice] A new release of pip available: 22.3.1 -> 23.0.1
[notice] To update, run: python3.9 -m pip install --upgrade pip


In [9]:
import jsonlines

dataset_file = "github-typo-corpus.v1.0.0.jsonl"

dataset = []
other_langs = set()

with jsonlines.open(dataset_file) as reader:
    for obj in reader:
        for edit in obj['edits']:
            if edit['src']['lang'] != 'eng':
                other_langs.add(edit['src']['lang'])
                continue

            if edit['is_typo']:
                src, tgt = edit['src']['text'], edit['tgt']['text']
                if src.lower() != tgt.lower():
                    dataset.append((edit['src']['text'], edit['tgt']['text']))
                
print(f"Dataset size = {len(dataset)}")

Dataset size = 245909


#### Explore sample typos
Please, explore the dataset. You may see, that this is
- mostly markdown
- some common mistakes with do/does
- some just refer to punctuation typos (which we do not consider)

In [10]:
for pair in dataset[1010:1020]:
    print(f"{pair[0]} => {pair[1]}")

        """Make am instance. =>         """Make an instance.
* travis: test agains Node.js 11 => * travis: test against Node.js 11
The parser receive a string and returns an array inside a user-provided  => The parser receives a string and returns an array inside a user-provided 
CSV data is send through the `write` function and the resulted data is obtained => CSV data is sent through the `write` function and the resulting data is obtained
One useful function part of the Stream API is `pipe` to interact between  => One useful function of the Stream API is `pipe` to interact between 
source to a `stream.Writable` object destination. This example available as  => source to a `stream.Writable` object destination. This example is available as 
`node samples/pipe.js` read the file, parse its content and transform it. => `node samples/pipe.js` and reads the file, parses its content and transforms it.
Most of the generator is imported from its parent project [CSV][csv] in a effort  => Most o

#### 1.2.0.1. Build a dataset vocabulary
Here we prepare a vocabulary for spellchecker testing and for estimating overall correction quality. Consider only word-level. Be carefull, there is markdown (e.g. \`name\`. \[url\]\(http://url)) and comment symbols (\#, //, \*).

In [11]:
def sent_to_words(sent):
    words = nltk.word_tokenize(sent)    
    words_filtered = filter(lambda x: x.isalpha(), words)
    return words_filtered

In [12]:
from collections import Counter
vocabulary = Counter()
for pair in dataset:
    for word in sent_to_words(pair[1].lower()):
        vocabulary[word] += 1
len(vocabulary)

63724

In [13]:
from itertools import islice
print(list(islice(vocabulary.items(), 10)))

[('function', 6193), ('de', 82), ('deutsch', 4), ('nocomments', 2), ('you', 42075), ('can', 26027), ('disable', 532), ('comments', 360), ('for', 44756), ('the', 207017)]


### 1.2.1. Implement context-independent spellcheker (Trigrams with Jaccard coefficient) ##

In [14]:
def intersection_over_union(set1, set2):
    return len(set1.intersection(set2)) / len(set1.union(set2))
    

def fix_typo_kgram(word, k_gram_index) -> list:
    #TODO return best matches with respect to Jaccard index
    k = len(list(k_gram_index.keys())[0])    
    augmented_word = '$' + word + '$'
    if len(augmented_word) < k:
        return None
       
    word_k_grams = set(split_to_k_grams(augmented_word,k))
    candidates = set()
    # retrieve postings list for each k-gram
    for k_gram in word_k_grams:
        candidates.update(k_gram_index.get(k_gram, []))
    # back to list
    candidates = list(candidates)
    
    if not candidates:
        return [word]
    
    return sorted(candidates, 
                  key = lambda x: intersection_over_union(set(split_to_k_grams('$' + x + '$',k)), word_k_grams),
                  reverse=True)

### 1.2.2. Tests

In [15]:
# tests

k_gram_index_github = build_k_gram_index(vocabulary, 3)
print(fix_typo_kgram("enouh", k_gram_index_github)[:20])
assert "enough" in fix_typo_kgram("enouh", k_gram_index_github), "Assert k-gram failed"

['enough', 'eno', 'enought', 'endogenous', 'enoent', 'enospc', 'enomem', 'enosys', 'enormous', 'renounce', 'uh', 'en', 'exogenous', 'enormously', 'homogenous', 'env', 'hetrogenous', 'end', 'ent', 'duh']


## 1.3. [Extra tasks, for fun]

### 1.3.1. QWERTY - Editorial distance

Write the code to compute weighted QWERTY-editorial distance.

<img src="https://upload.wikimedia.org/wikipedia/commons/thumb/d/da/KB_United_States.svg/640px-KB_United_States.svg.png" width="640"/> 

Use this image to prepare weight function:
- all letter pairs which share vertical border will get 0.5 multiplier **for replace** (`df`, `cv`, `ui`, ...)
- all letter pairs which share at least some horizontal border will get 0.7 multiplier **for replace** (`dc`, `dr`, `km`, ...)
- other operations are not scaled (x1 multiplier).

In [ ]:
def replace_weight(let1, let2) -> float:
    # TODO what is the weight for a pair of letters being replaced? 
    # Note this function should be commutative

    return 0.0

def qwerty_edit_dist(s1, s2) -> float:
    # TODO compute the Damerau-Levenshtein distance 
    # between two given strings (s1 and s2)

    return 0.0

In [ ]:
# tests

assert qwerty_edit_dist("korrectud", "corrected") == 2.0, "Edit distance is computed incorrectly"
assert qwerty_edit_dist("soem", "some") == 1.0, "Edit distance is computed incorrectly"
assert qwerty_edit_dist("one", "one") == 0.0, "Edit distance is computed incorrectly"
assert qwerty_edit_dist("fihure", "figure") == 0.5, "Edit distance is computed incorrectly"
assert qwerty_edit_dist("fivure", "figure") == 0.7, "Edit distance is computed incorrectly"

### 1.3.2. Norvig's spellchecker with QWERTY weights

You can base your code on [Norvig's corrector](https://norvig.com/spell-correct.html), but you should be sure you account the fact, that typos for close letters cost less. This should be considered in ranking.

In [ ]:
def fix_typo_qwerty_norvig(word) -> str:
    #TODO return best matching result for the word
    
    return ""

In [ ]:
# tests

assert fix_typo_qwerty_norvig("korrectud") == "corrected", "Norvig's correcter doesn't work"
assert fix_typo_qwerty_norvig("speling") == "spelling", "Norvig's correcter doesn't work"
assert fix_typo_qwerty_norvig("condidence") == "confidence", "Norvig's correcter doesn't work"
assert fix_typo_qwerty_norvig("fpx") == "fox", "Norvig's correcter doesn't work"
assert fix_typo_qwerty_norvig("fux") == "fix", "Norvig's correcter doesn't work"

### 12.3.3. Estimate quality of functions

In [ ]:
norvig, kgram = 0, 0
limit = 10000
counter = limit
for i, (src, target) in enumerate(dataset):
    if i == limit:
        break
    words = sent_to_words(src.lower())
    # word suspected for typos
    sn, sk = src.lower(), src.lower()
    for word in words:
        if word not in vocabulary and word.isalpha():
            # top-1 accuracy
            wn, wk = fix_typo_qwerty_norvig(word), \
                     fix_typo_kgram(word, k_gram_index_github)[0]
            sn = sn.replace(word, wn)
            sk = sk.replace(word, wk)
    norvig += int(sn == target.lower())
    kgram += int(sk == target.lower())

print(f"Norvig accuracy ({norvig}) = {norvig / limit}")
print(f"k-gram accuracy ({kgram}) = {kgram / limit}")